## Setup

In [ ]:
# 1) Clone & cd
!git clone https://github.com/alonlahav123/DSC291-CLIP-Dissect.git
%cd DSC291-CLIP-Dissect

# # 2) Install PyTorch & torchvision
# !pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu117

# 3) Install other deps
!pip install -r requirements.txt

# 3b) install these again (not sure why but this works)
!pip install tokenizer ftfy sentence-transformers huggingface-hub

# 4) Download the Broden dataset (images only) using
!bash dlbroden.sh

Cloning into 'DSC291-CLIP-Dissect'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 116 (delta 40), reused 115 (delta 39), pack-reused 0 (from 0)
Receiving objects: 100% (116/116), 15.95 MiB | 17.52 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/DSC291-CLIP-Dissect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 65.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.3 M

In [1]:
%%bash
# 1) upload imagenet_val 2012 after downloading from imagenet website
# 2) extract + remove the .tar.gz to save space

## Imagenet_val setup

In [ ]:
%%bash
cd data/imagenet_val

# Create the devkit folder if it doesn’t exist
mkdir -p ILSVRC2012_devkit_t12/data

# Download a known‐good meta.mat
wget -O ILSVRC2012_devkit_t12/data/meta.mat \
  https://raw.githubusercontent.com/calebrob6/imagenet_validation/master/data/meta.mat

# Download the ground-truth labels (50 000 integers 1–1000)
wget -O ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt \
  https://raw.githubusercontent.com/calebrob6/imagenet_validation/master/data/ILSVRC2012_validation_ground_truth.txt

# Need to organize the 50000 images into 1000 class folders (downloaded dataset is a flat folder with all the images)
python3 <<'EOF'
import os, numpy as np, scipy.io, shutil

# 1) Hardcode the full paths to the new files:
meta_mat_path = os.path.expanduser('~/private/DSC291-CLIP-Dissect/data/imagenet_val/ILSVRC2012_devkit_t12/data/meta.mat')
gt_path       = os.path.expanduser('~/private/DSC291-CLIP-Dissect/data/imagenet_val/ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt')

# 2) Where your flat JPEGs live and where to write val_annotations.txt
flat_dir  = 'data/imagenet_val/images'
anns_file = 'data/imagenet_val/val_annotations.txt'

# 3) Load the 1 000 WNIDs from meta.mat
mat = scipy.io.loadmat(meta_mat_path, struct_as_record=False, squeeze_me=True)
wnids = [str(s.WNID) for s in mat['synsets']]

# 4) Load the 50 000 labels (values 1–1000)
gt = np.loadtxt(gt_path, dtype=int)

# 5) Sort the 50 000 flat JPEG filenames (match .JPEG or .jpeg)
imgs = sorted(f for f in os.listdir(flat_dir) if f.lower().endswith('.jpeg'))

# 6) Write val_annotations.txt linking each image to its WNID
with open(anns_file, 'w') as out:
    for img, lbl in zip(imgs, gt):
        out.write(f"{img}\t{wnids[lbl-1]}\n")

# 7) Move each JPEG into its synset subfolder
for line in open(anns_file):
    img, wnid = line.strip().split('\t')
    src = os.path.join(flat_dir, img)
    dst = os.path.join(flat_dir, wnid)
    os.makedirs(dst, exist_ok=True)
    shutil.move(src, os.path.join(dst, img))

print("✅ Done. Now have", len(os.listdir(flat_dir)), "folders under", flat_dir)
EOF


# Check to see that all the images are classfiied. 
ls data/imagenet_val/images | wc -l   # should print “1000”

## Run CLIP Dissect

In [2]:
!python describe_neurons.py \
  --clip_model ViT-B/16 \
  --target_model resnet50 \
  --d_probe imagenet_broden \
  --concept_set data/20k.txt \
  --pool_mode avg \
  --similarity_fn soft_wpmi \
  --batch_size 200 \
  --device cuda

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /tmp/xdg-cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████████████████████████████████| 97.8M/97.8M [00:01<00:00, 79.5MB/s]
100%|██████████████████████████████████████████| 64/64 [00:00<00:00, 132.16it/s]
torch.Size([64, 20000])
100%|████████████████████████████████████████| 256/256 [00:02<00:00, 123.31it/s]
torch.Size([256, 20000])
100%|████████████████████████████████████████| 512/512 [00:03<00:00, 131.48it/s]
torch.Size([512, 20000])
100%|██████████████████████████████████████| 1024/1024 [00:07<00:00, 137.65it/s]
torch.Size([1024, 20000])
100%|██████████████████████████████████████| 2048/2048 [00:15<00:00, 134.96it/s]
torch.Size([2048, 20000])
